<h1 style="text-align: center;"><b>Практические примеры использовния статистических тестов
</b></h1>

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as st
import seaborn as sns
import datetime
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
import warnings

sns.set_style("whitegrid")
pd.options.display.max_colwidth = 18
pd.options.display.max_columns = 50
pio.templates.default = 'plotly_white'

## 1. Сравнение эффетиквности стратегий в онлайн магазине

Компания представила новую стратегию рекомендаций для своего онлайн магазина. Было решено провести A/B тест для сравнения эффективности новой стратегии с предыдущей. Тест выполнялся в течение одного месяца, были собранны данные от 30 клиентов по каждой стратегии и теперь их необходимо проанализировать.

#### Импорт данных

In [ ]:
df_control = pd.read_csv('control_group.csv', sep = ";")
df_control.head()

Иногда для удобства можно переименовать столбцы

In [ ]:
df_control.rename(columns = {'Name':'Имя', 'Date': 'Дата','Spent [USD]': 'Расход USD',\
                             '# of Impressions':'Кол-во показов','Reach':'Охват','# of Clicks':'Кол-во кликов',\
                             '# of Searches':'Кол-во запросов','# of Views':'Кол-во просмотров',
                             '# of Add to Cart':'В корзине','# of Purchase':'Кол-во покупок'}, inplace = True)

In [ ]:
df_control.info()

In [ ]:
df_test = pd.read_csv('test_group.csv', sep = ";")
df_test.head()

In [ ]:
df_test.rename(columns = {'Name':'Имя', 'Date': 'Дата','Spent [USD]': 'Расход USD',\
                             '# of Impressions':'Кол-во показов','Reach':'Охват','# of Clicks':'Кол-во кликов',\
                             '# of Searches':'Кол-во запросов','# of Views':'Кол-во просмотров',
                             '# of Add to Cart':'В корзине','# of Purchase':'Кол-во покупок'}, inplace = True)

In [ ]:
df_test.info()

#### Обработка пропусков в данных

1. Если пропусков мало, то такие строки можно просто удалить

In [ ]:
df_drop = df_control.dropna().reset_index(drop = True)

In [ ]:
df_drop.describe()

2. Но если данных мало, то стоит поискать альтернативные варианты - например, заполнение похожими значениями

In [ ]:
from sklearn.impute import KNNImputer

In [ ]:
imputer = KNNImputer(weights='distance')

In [ ]:
columns_to_impute = df_control.columns[2:10]
imputed_data = imputer.fit_transform(df_control[columns_to_impute])

imputed_df = pd.DataFrame(imputed_data, columns=columns_to_impute)

In [ ]:
df_control[columns_to_impute] = imputed_df

In [ ]:
df_control.describe()

#### Подготовка данных к анализу

Объединим данные в один фрейм и рассчитаем несколько ключевых характеристик

In [ ]:
df = pd.concat([df_test,df_control])
df = df.reset_index()

df['USD за покупку'] = round(df['Расход USD']/df['Кол-во покупок'],2)

df['CTR(%)'] = round(df['Кол-во кликов']*100/df['Кол-во показов'],2)

df['Конверсия(%)'] = round(df['Кол-во покупок']*100/df['Кол-во кликов'],2)

In [ ]:
df = df.drop(['index'], axis=1)
df

#### Визуализация данных

Целью данного этапа яявляется проверка адекватности собранных данных, а также выявление идей для дальнейшего анализа

In [ ]:
sns.set_theme(style="ticks")
g = sns.jointplot(
    data=df,
    x="CTR(%)", y="Конверсия(%)", hue="Имя",
    kind="kde",
)
plt.show()

In [ ]:
sns.set_theme(style="ticks")
g = sns.jointplot(
    data=df,
    x="Кол-во кликов", y="Кол-во покупок", hue="Имя",
    kind="kde",
)
plt.show()

In [ ]:
sns.set_theme(style="ticks")
g = sns.jointplot(
    data=df,
    x="В корзине", y="Кол-во покупок", hue="Имя",
    kind="kde",
)
plt.show()

In [ ]:
sns.set_theme(style="ticks")
g = sns.jointplot(
    data=df,
    x="Расход USD", y="USD за покупку", hue="Имя",
    kind="kde",
)
plt.show()

In [ ]:
sns.set_theme(style="ticks")
g = sns.jointplot(
    data=df,
    x="Кол-во показов", y="Охват", hue="Имя",
    kind="kde",
)
plt.show()

#### Проверка гипотез

<b>Идея 1.</b> Проверка новой и старой кампаний на предмет эффективности расходов. 

In [ ]:
total_spend = df.groupby(['Имя'])['Расход USD'].sum().reset_index()
total_spend

In [ ]:
spend_per_purchase = df.groupby(['Имя'])['USD за покупку'].mean().reset_index()
spend_per_purchase

Из цифр видно, что тестовая компания требует больших расходов, но является ли эта разница статистически значимой или скорее случайной?
Чтобы ответить на этот вопрос, проведем статистический тест. 

Для начала определимся с распределением данных

In [ ]:
data_1 = df.loc[df['Имя']=='Control Campaign']['USD за покупку'].values
data_2 = df.loc[df['Имя']=='Test Campaign']['USD за покупку'].values

In [ ]:
sns.kdeplot(data_1, label="Control", fill = True, color = '#000000')
sns.kdeplot(data_2, label="Test", fill = True, color = '#B22222')
plt.legend()
plt.show()

In [ ]:
stat, p = st.shapiro(data_1)

print(f'stat = {stat:.3f}, p = {p:.5f}')
if p > 0.05:
    print('Вероятно нормальное распределение')
else:
    print('Вероятно не нормальное распределение')

In [ ]:
stat, p = st.shapiro(data_2)

print(f'stat = {stat:.3f}, p = {p:.5f}')
if p > 0.05:
    print('Вероятно нормальное распределение')
else:
    print('Вероятно не нормальное распределение')

Распределение ненормальное, поэтому будем использовать критерий Манна-Уитни

In [ ]:
stat, p = st.mannwhitneyu(data_1, data_2)

print(f"Статистика = {stat:.5f}, p = {p:.5f}")

if p > 0.05:
    print('Не отклоняем нулевую гипотезу, распределения, вероятно, одинаковые')
else:
    print('Отклоняем нулевую гипотезу, распределения, вероятно, различаются')

<b>Вывод.</b> Тест не показал наличие статистически значимой разницы в расходах - при текущих данных мы не можем утверждать, что тестовая компания будет обходится нам дороже. 

<b>Идея 2.</b> Проверка новой и старой кампаний на предмет увеличения числа покупок. 

In [ ]:
num_of_purchase = df.groupby(['Имя'])['Кол-во покупок'].sum().reset_index()
num_of_purchase

Цифры показывают незначительное падение продаж для новой кампании, но является ли оно статистически значимым?

In [ ]:
data_1 = df.loc[df['Имя']=='Control Campaign']['Кол-во покупок'].values
data_2 = df.loc[df['Имя']=='Test Campaign']['Кол-во покупок'].values

In [ ]:
sns.kdeplot(data_1, label="Control", fill = True, color = '#000000')
sns.kdeplot(data_2, label="Test", fill = True, color = '#B22222')
plt.legend()
plt.show()

In [ ]:
stat, p = st.shapiro(data_1)

print(f"Статистика = {stat:.5f}, p = {p:.5f}")
if p > 0.05:
    print('Вероятно нормальное распределение')
else:
    print('Вероятно не нормальное распределение')

In [ ]:
stat, p = st.shapiro(data_2)

print(f"Статистика = {stat:.5f}, p = {p:.5f}")
if p > 0.05:
    print('Вероятно нормальное распределение')
else:
    print('Вероятно не нормальное распределение')

Распределение ненормальное, поэтому будем использовать критерий Манна-Уитни

In [ ]:
stat, p = st.mannwhitneyu(data_1, data_2)

print(f"Статистика = {stat:.5f}, p = {p:.5f}")

if p > 0.05:
    print('Не отклоняем нулевую гипотезу, распределения, вероятно, одинаковые')
else:
    print('Отклоняем нулевую гипотезу, распределения, вероятно, различаются')

<b>Вывод.</b> Тест не показал наличие статистически значимой разницы в продажах - при текущих данных мы не можем утверждать, что тестовая компания уменьшает или увеличивает продажи. 

<b>Идея 3.</b> Проверка новой и старой кампаний на предмет увеличения конверсии клиентов. 

In [ ]:
df_2 = df.copy()
df_2 = df_2.drop(['Дата'], axis=1)
df_2['Имя'] = pd.get_dummies(df_2['Имя'], drop_first=True)
df_2 = df_2.reset_index(drop=True)
test_g = df_2[df_2['Имя']==1]
control_g = df_2[df_2['Имя']==0]

In [ ]:
test_group = test_g['Конверсия(%)']
test_group.describe()

In [ ]:
control_group = control_g['Конверсия(%)']
control_group.describe()

In [ ]:
sns.kdeplot(control_group, label="Control", fill = True, color = '#000000')
sns.kdeplot(test_group, label="Test", fill = True, color = '#B22222')
plt.legend()
plt.show()

In [ ]:
sns.boxplot(df['Конверсия(%)']);

Наблюдаем несколько необычно больших значений конверсии. Такие события происходят редко, поэтому мы можем исключить их из рассмотрения.

In [ ]:
print(np.where(df['Конверсия(%)']>25))

In [ ]:
df_2 = df_2[df_2['Конверсия(%)']<25]
test_g = df_2[df_2['Имя']==1]
control_g = df_2[df_2['Имя']==0]
test_group = test_g['Конверсия(%)']
control_group = control_g['Конверсия(%)']

In [ ]:
sns.kdeplot(control_group, label="Control", fill = True, color = '#000000')
sns.kdeplot(test_group, label="Test", fill = True, color = '#B22222')
plt.legend()
plt.show()

In [ ]:
stat, p = st.shapiro(test_group)

print(f"Статистика = {stat:.5f}, p = {p:.5f}")
if p > 0.05:
    print('Вероятно нормальное распределение')
else:
    print('Вероятно не нормальное распределение')

In [ ]:
stat, p = st.shapiro(control_group)

print(f"Статистика = {stat:.5f}, p = {p:.5f}")
if p > 0.05:
    print('Вероятно нормальное распределение')
else:
    print('Вероятно не нормальное распределение')

Используем критерий Манна-Уитни. 

In [ ]:
stat, p = st.mannwhitneyu(control_group, test_group)

print(f"Статистика = {stat:.5f}, p = {p:.5f}")

if p > 0.05:
    print('Не отклоняем нулевую гипотезу, распределения, вероятно, одинаковые')
else:
    print('Отклоняем нулевую гипотезу, распределения, вероятно, различаются')

А что если использовать тест Стъюдента, ведь распределения данных близк к нормальным. Для этого необходимо вначале проверить равенство вариаций. 

In [ ]:
stat, p = st.levene(control_group, test_group)

print(f"Статистика = {stat:.5f}, p = {p:.5f}")

if p > 0.05:
    print("Не отклоняем нулевую гипотезу >> Вариация в группах одинаковая")
else:
    print("Отклоняем нулевую гипотезу >> Вариация в группах различается")

In [ ]:
stat, p = st.ttest_ind(control_group, test_group)

print(f"Статистика = {stat:.5f}, p = {p:.5f}")

if p > 0.05:
    print('Не отклоняем нулевую гипотезу, распределения, вероятно, одинаковые')
else:
    print('Отклоняем нулевую гипотезу, распределения, вероятно, различаются')

<b>Вывод.</b> Тест не показал наличие статистически значимой разницы в конверсии. На основе трех проведенных исследований можно утверждать, что новая кампания не позволяет улучшить показатели ни по одному из трех критериев. 

<b>Анализ трендов.</b> Наличие или отстутсвие эффекта при текущих данных не означает сохранение эффекта в будущем. Для простого анализа тенденций можно использовать линейные регрессионные модели. 

In [ ]:
warnings.filterwarnings("ignore", category = FutureWarning, module="plotly")

In [ ]:
fig = px.scatter(data_frame=df, 
                 x = 'Кол-во показов',
                 y = 'Расход USD',
                 size = 'Расход USD',
                 color = 'Имя',
                 trendline = 'ols'
                )
fig.show()

In [ ]:
fig = px.scatter(df, x='Кол-во показов', 
                 y='Кол-во кликов', 
                 size='Кол-во кликов', 
                 color='Имя', trendline='ols')
fig.show()

In [ ]:
fig = px.scatter(df, x='В корзине', 
                 y='Кол-во покупок', 
                 size='Кол-во покупок', 
                 color='Имя', trendline='ols')
fig.show()

## 2. Анализ качества продукции в фармацевтической компании

Фармацевтическая компания обеспокоена качеством изготавливаемой продукции на разных производственных линиях. Руководство хочет сравнить продукцию, выпускаемую в разные смены на разных линиях на предмет концентрации активного ингридиента в таблетках как с точки зрения среднего значения, так и по уровню вариации. 

Критическим считается различием более чем в одну $\sigma$ (стандартное отклонение).  Для достоверности исследования мощность должна составлять не менее $80\%$

In [ ]:
import numpy as np
import scipy.stats as st
import statsmodels.api
import pandas as pd
import statsmodels.api as sm
import pylab as py
from statsmodels.stats.power import FTestAnovaPower
from statsmodels.stats.oneway import effectsize_oneway

#### Анализ мощности

Сбор данных на производстве может быть дорогостоящим, поэтому необходимо заранее понимать какая точность анализа необходима и сколько нужно собрать данных для достижения этой точности. 

Сначала необходимо определить величину эффекта - какую разницу мы хотим, чтобы наш тест обнаружил. 

In [ ]:
nobs = np.array([10, 10, 10])
delta = 1.5 
means_alt = np.array([-1, 0, 1]) * delta
vars_ = np.arange(1, len(means_alt) + 1)

effect = effectsize_oneway(means_alt, vars_, nobs, use_var="equal")
effect

Теперь определим требуемый размер выборки

In [ ]:
alpha = 0.05
power = 0.80

analysis = FTestAnovaPower()
result = np.ceil(analysis.solve_power(effect, power=power, nobs=None, alpha=alpha, k_groups=3))
print(f"Необходимый размер выборки: {result:.0f}")

In [ ]:
from numpy import array
from matplotlib import pyplot

effect_sizes = array([effect])
sample_sizes = array(range(10, 40))

analysis = FTestAnovaPower()
analysis.plot_power(dep_var='nobs', nobs=sample_sizes, effect_size=effect_sizes)
pyplot.show()

Также можно построить обратный график - зависимость мощности от величины эффекта

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from statsmodels.stats.power import FTestAnovaPower

# Define the effect sizes and sample size
effect_sizes = np.linspace(0.01, 1.0, num=50)  # Example effect sizes
sample_size = 30  # Example sample size

fig, ax = plt.subplots()

# Create an instance of the FTestAnovaPower class
analysis = FTestAnovaPower()

# Initialize lists to store effect sizes and power values
effect_list = []
power_list = []

# Calculate power for each effect size
for effect in effect_sizes:
    power = analysis.power(effect_size=effect, nobs=sample_size, alpha = alpha)
    effect_list.append(effect)
    power_list.append(power)

# Plot a joined line connecting the points
ax.plot(effect_list, power_list, linestyle='-', label='Power vs Effect Size')

# Add labels and legend
ax.set_xlabel('Effect Size')
ax.set_ylabel('Power')
ax.set_title('Power Analysis')

# Show the plot
plt.show()


Теперь когда мы знаем сколько данных нам необходимо, давайте из соберем и проанализируем. 

#### Импорт данных

In [ ]:
df = pd.read_excel('actIngrid.xlsx')
df.head()

In [ ]:
df.info()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.relplot(data=df, x=df['Shift'], y=df['Ingredient']);

In [ ]:
sns.relplot(data=df, x=df['Shift'], y=df['Ingredient'], kind = "line");

Графики показывают, что есть заметная разница между средними значениями трех групп. Определим, является ли данная разница статистически значимой. 

#### Дисперсионный анализ (ANOVA) 

In [ ]:
data_1 = df.loc[df['Shift']=='A']['Ingredient'].values
data_2 = df.loc[df['Shift']=='B']['Ingredient'].values
data_3 = df.loc[df['Shift']=='C']['Ingredient'].values

Проверим распределения данных с помощью графика и с помощью тестов. 

In [ ]:
res1 = st.probplot(data_1, plot=plt)

In [ ]:
stat, p = st.shapiro(data_1)

print(f"Статистика = {stat:.5f}, p = {p:.5f}")

if p > 0.05:
    print('Вероятно нормальное распределение')
else:
    print('Вероятно не нормальное распределение')

In [ ]:
res2 = st.probplot(data_2, plot=plt)

In [ ]:
stat, p = st.shapiro(data_2)

print(f"Статистика = {stat:.5f}, p = {p:.5f}")

if p > 0.05:
    print('Вероятно нормальное распределение')
else:
    print('Вероятно не нормальное распределение')

In [ ]:
res3 = st.probplot(data_3, plot=plt)

In [ ]:
stat, p = st.shapiro(data_3)

print(f"Статистика = {stat:.5f}, p = {p:.5f}")

if p > 0.05:
    print('Вероятно нормальное распределение')
else:
    print('Вероятно не нормальное распределение')

Прежде чем переходить к дисперсионному анализу, проверим равенство вариаций. 

In [ ]:
stat, p = st.levene(data_1,data_2,data_3)

print(f"Статистика = {stat:.5f}, p = {p:.5f}")

if p <0.05:
    print("Отклоняем нулевую гипотезу >> Вариация в группах раличается")
else:
    print("Не отклоняем нулевую гипотезу >> Вариация в группах одинаковая")

Теперь используем однофакторный дисперсионный анализ.

In [ ]:
fvalue, pvalue = st.f_oneway(data_1,data_2,data_3)

print(f"Статистика = {fvalue:.5f}, p = {pvalue:.5f}")

if pvalue > 0.05:
    print('Не отклоняем нулевую гипотезу, средние, вероятно, одинаковые')
else:
    print('Отклоняем нулевую гипотезу, средние, вероятно, различаются')

То же самое, но более профессионально. Подход с использованием обощенных линейных моделей позволяет решать более продвинутые задачи и моделировать более сложные процессы. Для успешного использования необходимы углубленные знания статистики. Иногда продвинутые модели легче использовать в специальных статистических пакетах, например Minitab. 

In [ ]:
from statsmodels.formula.api import ols

In [ ]:
model = ols('Ingredient ~ C(Shift)',
            data=df).fit()
result = sm.stats.anova_lm(model, type=1)
  
# Print the result
print(result)

In [ ]:
print(str(model.summary()))

После обнаружения статистически значимого эффекта необходимо дополнительное исследование с учетом проблемы множественного сравнения. 

#### Множественное сравнения - критерий Тьюки

In [ ]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

tukey = pairwise_tukeyhsd(endog=df['Ingredient'].values,
                          groups=df['Shift'],
                          alpha=0.05)
print(tukey)

In [ ]:
tukey.plot_simultaneous(comparison_name="B");

#### Выводы

Мы обнаружили, что одна из линий статистически отличается по концетрации активного вещества от двух других в меньшую сторону. Это может говорить об ухудшении качества продукции. 

При сравнении числа групп больше 2 использование дисперсионного анализа является предпочтительным по причине неточностей, возникающих при множественном сравнении. В общем случае нормальность данных неявляется критическим условием при больших размерах выборки. 

## 3. Оптимизаци маркетинговой стратегии

Компания анализирует свои расходы на рекламу с целью оптимизации маркетинговой стратегии. У нас есть данные по двум группам людей - кому показали рекламу (ad) и кому показали просто публичное объявление (psa). Для каждого учатсника эксперимента оценивается конверсия. Дополнительная информация:
* Total ads. Сколько раз человек видел рекламу/объявление.
* Most ads day. В какой день человек видел наибольшее число рекламы/объявлений.
* Most ads hour. В какой час человек видел наибольшее число рекламы/объявлений.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from statsmodels.stats import power
from statsmodels.stats.proportion import proportions_ztest
sns.set_style("whitegrid")

In [ ]:
df = pd.read_csv('marketing.csv')
df.head()

In [ ]:
df.info()

#### Подготовка данных

Удалим дублирующий первый столбец

In [ ]:
df = df.drop(columns = ['Unnamed: 0'])
df.head()

Иногда для удобства можно автоматически превращать имена столбцов в одной слово, заменяя пробелы на подчеркивания

In [ ]:
df.columns = df.columns.str.replace(' ', '_')
df.head()

Проверка на наличие дублирующихся записей

In [ ]:
df[df.duplicated()]

Проверка на наличие пропусков

In [ ]:
df.isna().sum()

Проверка адекватности данных с помощью описательной статистики

In [ ]:
df.total_ads.describe()

In [ ]:
sns.histplot(df.total_ads)
plt.title('Распределение числа показов рекламы/объявлений')
plt.show()

Показ рекламы более 2000 раз в день - явная ошибка. Необходимо подчистить выбросы

In [ ]:
df[df.total_ads > df.total_ads.quantile(q=0.97)].groupby('test_group')['total_ads'].agg(['mean', 'min', 'max'])

In [ ]:
df2 = df[df.total_ads < df.total_ads.quantile(q=0.99)]
df2.total_ads.describe()

In [ ]:
sns.histplot(df2.total_ads)
plt.title('Распределение числа показов рекламы/объявлений')
plt.show()

Проверка адекватности категориальных данных

In [ ]:
for col in ['test_group', 'converted', 'most_ads_day', 'most_ads_hour']:
    print(col, 'distribution')
    print(df2[col].value_counts())
    print()

Расчет общей конверсии

In [ ]:
df2.converted.value_counts(normalize=True)

Конверсия внутри каждой группы

In [ ]:
df2.groupby('test_group')['converted'].mean()

In [ ]:
ax = sns.countplot(x="converted", hue="test_group", data=df)

counts = df.groupby(["test_group", "converted"]).size().reset_index(name="count")

for i, bar in enumerate(ax.containers):
    for j, count in enumerate(bar):
        height = count.get_height()
        ax.annotate(counts.iloc[i*2+j]["count"], xy=(count.get_x() + count.get_width() / 2, height),
                    xytext=(0, 3), textcoords="offset points", ha="center", va="bottom")

plt.show()

In [ ]:
ax = sns.barplot(x=df2.groupby('test_group')['converted'].mean().index,
            y=df2.groupby('test_group')['converted'].mean()*100)

rects = ax.patches
labels = [str(i) + ' %' for i in round(df2.groupby('test_group')['converted'].mean()*100, 2)]

for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax.text(
        rect.get_x() + rect.get_width() / 2, height, label, ha="center", va="bottom"
    )
plt.title('Конверсия в разных группах', va='bottom' )
plt.ylabel('Коэффициент конверсии')
plt.show()

Размер эффекта между двумя пропорциями (Cohen's h)

In [ ]:
test = df2.test_group == 'ad'
control = df2.test_group == 'psa'

n_control = len(df2[control])
n_test = len(df2[test])
p_control = round(df2[control]['converted'].mean(),4)
p_test = round(df2[test]['converted'].mean(),4)
print("Размер контрольной группы -", n_control)
print("Размер тестовой группы -", n_test)
print("Пропорция (конверсия) контрольной группы -", p_control)
print("Пропорция (конверсия) тестовой группы -", p_test)

In [ ]:
from statsmodels.stats.proportion import proportion_effectsize
effect_size = proportion_effectsize(p_control, p_test)

delta_p = p_test - p_control
print('Абсолютная разница в пропорциях: ', abs(round((delta_p)*100,2)), '%')
print('Размер эффекта:', abs(round(effect_size,2)))

if effect_size > 0.8:
  print('Размер эффекта большой')
elif effect_size > 0.5:
  print('Размер эффекта средний')
elif effect_size > 0.2:
  print('Размер эффекта маленький')
else:
  print('Размер эффекта очень маленький')

Важно проверить качество обеих групп. Нет ли критических различий между группами, которые бы могли влиять на результаты эксперимента, мешая понять обособленную роль рекламы. В первую очередь проверим число показа рекламы/объявлений по группам.

In [ ]:
df2.groupby('test_group')['total_ads'].mean()

In [ ]:
sns.boxplot(data=df2, y='test_group', x='total_ads', showmeans=True);

Затем проверим время показа и расперделение по дням и по часам внутри каждого дня

In [ ]:
round(pd.crosstab(df.test_group, df.most_ads_day, normalize='index') * 100, 2)

In [ ]:
round(pd.crosstab(df.test_group, df.most_ads_hour, normalize='index') * 100, 2)

Расчет минимально необходимого числа замеров

In [ ]:
alpha = 0.05
beta = 0.1

sample_size = power.TTestIndPower().solve_power(nobs1=None, effect_size=effect_size, alpha=alpha, power=1-beta)
n_required = round(sample_size)
print(n_required)

Для начала проведем тест с необходимым размером выборки. 

Двусторонний тест:

Нулевая гипотеза Hₒ: p2 - p1 = 0 .
Альтернативная гипотеза Hₐ: p2 - p1 ≠ 0

In [ ]:
df2[df2.test_group == 'ad']['converted'].sample(n=n_required, random_state=42).sum()/n_required

In [ ]:
df2[df2.test_group == 'psa']['converted'].sample(n=n_required).sum()/n_required

In [ ]:
test = df2[df2.test_group == 'ad']['converted'].sample(n=n_required).sum()/n_required
control = df2[df2.test_group == 'psa']['converted'].sample(n=n_required).sum()/n_required

z_stat, p_value = proportions_ztest([control, test], nobs=[n_required, n_required], alternative='two-sided')

print(f'z = {z_stat}')
print(f'p-value = {p_value}')

if p_value > 0.05:
    print('Не отклоняем нулевую гипотезу, пропорции, вероятно, одинаковые')
else:
    print('Отклоняем нулевую гипотезу, пропорции, вероятно, различаются')

А что если использовать все данные? Наш размер превосходит достаточный для определение данного эффекта. В очень много раз. Это значит, что есть большой шанс получить разницу, которая статистичски значимая, но практически - незначимая.

In [ ]:
test = df2[df2.test_group == 'ad']['converted'].sum() 
control = df2[df2.test_group == 'psa']['converted'].sum()

z_stat, p_value = proportions_ztest([control, test], nobs=[n_control, n_test] ,
                                  alternative='two-sided')
print(f'z = {z_stat}')
print(f'p-value = {p_value}')

if p_value > 0.05:
    print('Не отклоняем нулевую гипотезу, пропорции, вероятно, одинаковые')
else:
    print('Отклоняем нулевую гипотезу, пропорции, вероятно, различаются')

In [ ]:
p_pooled = round((n_control * p_control + n_test * p_test)/(n_control + n_test),3)
se = round(np.sqrt(p_pooled*(1 - p_pooled) * (1/n_control + 1/n_test)),3)
mu_delta = 0
cutoff_left = round(norm.ppf(alpha/2, loc=mu_delta, scale=se),3)
cutoff_right = round(norm.ppf(1 - alpha/2, loc=mu_delta, scale=se),3)
delta = p_test - p_control

print('Общая объединенная доля =', p_pooled)
print('Общая объединенная стандартная ошибка =', se)
print('Критические значения = ', cutoff_left, 'и', cutoff_right)
print('Наблюдаемая разница =', delta)

In [ ]:
# График распределения
fig, ax = plt.subplots()


x = np.linspace( mu_delta - 8.5*se, mu_delta + 8.5*se, 1000)
y = norm.pdf(x, loc=mu_delta, scale= se)
sns.lineplot(x=x, y=y, ax=ax)

#mu delta
ax.axvline(mu_delta, linestyle='--')

#x ticks
plt.xticks([delta, cutoff_left, mu_delta, cutoff_right])
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.plot(delta, 0, "b^", markersize=15)
plt.annotate("Разница", xy=(0.0079, 10), xytext=(0.0055, 95), 
            arrowprops={"arrowstyle":"->", "color":"gray"},
             fontsize=10)

#alpha
alpha_x = np.linspace(cutoff_right, mu_delta + 8.5 * se, 1000)
alpha_y = norm.pdf(alpha_x, loc = mu_delta, scale= se)
plt.fill_between(alpha_x, alpha_y, color='red', alpha=0.5)

alpha_x2 = np.linspace(mu_delta - 8.5 * se, cutoff_left, 1000)
alpha_y2 = norm.pdf(alpha_x2, loc = mu_delta, scale= se)
plt.fill_between(alpha_x2, alpha_y2, color='red', alpha=0.5)

plt.text(-0.0055, 20, 'Alpha / 2', color='red', fontsize=10)
plt.text(0.003, 20, 'Alpha / 2', color='red', fontsize=10)

#H null
plt.text(0.003, 360, 'Ho: (p2 - p1) = 0')

#labels
plt.xlabel('Разница пропорций (Нулевая гипотеза)')
plt.ylabel('PDF')

plt.autoscale(enable=True, axis='both', tight=True) 


# Выводы

Нам удалось обнаружить статистически значимую разницу, но нужно учитывать, что такая разница достигнута за счет очень большого размера выборок. Такой результат на практике скорее всего не будет иметь значения.